<a href="https://colab.research.google.com/github/dede0702/mnist-digit-4-gan/blob/main/GAN_MNIST_Digito_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 ```
🧩 Etapa 1: Instalar dependências
 ```

In [22]:
!pip install torch torchvision matplotlib

 ```
📦 Etapa 2: Importar bibliotecas
 ```

In [23]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets # Alterado para torchvision
import matplotlib.pyplot as plt



```
📁 Etapa 3: Carregar apenas os dígitos 4 do MNIST (usando torchvision)
```



In [24]:
class MNIST4Dataset(Dataset):
    def __init__(self):
        # Carregar o dataset MNIST usando torchvision
        mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=None)

        # Filtrar para incluir apenas os dígitos '4'
        idx = mnist_trainset.targets == 4
        data_4_filtered = mnist_trainset.data[idx] # Tensor de formato (N, 28, 28), tipo uint8

        # Converter para float, normalizar para [-1, 1] e adicionar dimensão de canal
        # Imagens são (H, W), precisamos de (C, H, W) para Conv2d, então (1, 28, 28)
        data_normalized = data_4_filtered.float() / 255.0  # Escala para [0, 1]
        self.data = (data_normalized * 2.0) - 1.0      # Escala para [-1, 1]
        self.data = self.data.unsqueeze(1)   # Adiciona dimensão de canal: (N, 1, 28, 28)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

dataset = MNIST4Dataset()
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

Número de amostras do dígito 4: 5842


In [25]:
# Visualizar algumas imagens do dataset para confirmar
print(f"Número de amostras do dígito 4: {len(dataset)}")
fig, axs = plt.subplots(1, 5, figsize=(10, 2))
for i in range(5):
    img = dataset[i].squeeze().numpy() # Remove canal, converte para numpy
    axs[i].imshow(img, cmap='gray')
    axs[i].set_title(f"Amostra {i+1}")
    axs[i].axis('off')
plt.show()



```
🧠 Etapa 4: Definir o Discriminador
```



In [26]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1, 64, 4, 2, 1, bias=False), # Imagens 1x28x28
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3), # Adicionado Dropout após primeira conv como no original (mas antes não tinha no código)
            # Ação: 64 x 14 x 14
            nn.Conv2d(64, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128), # Adicionado BatchNorm para estabilidade
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            # Ação: 128 x 7 x 7
            nn.Flatten(),
            nn.Linear(128 * 7 * 7, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

# Observação: Adicionei bias=False nas camadas convolucionais e BatchNorm2d no Discriminador, que são práticas comuns em arquiteturas DCGAN e podem ajudar na estabilidade.
# O Dropout original foi mantido, embora sua posição possa variar.



```
🎨 Etapa 5: Definir o Gerador
```



In [27]:
class Generator(nn.Module):
    def __init__(self, latent_dim):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            # Entrada: latent_dim
            nn.Linear(latent_dim, 256 * 7 * 7),
            # Não há LeakyReLU/BatchNorm1d na versão original diretamente após o Linear antes do Unflatten
            # Mas o código fornecido tinha. Vou mantê-lo.
            nn.LeakyReLU(0.2, inplace=True),
            nn.BatchNorm1d(256 * 7 * 7),
            nn.Unflatten(1, (256, 7, 7)),
            # Ação: 256 x 7 x 7
            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False), # output 128 x 14 x 14
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            # Ação: 128 x 14 x 14
            nn.ConvTranspose2d(128, 1, 4, 2, 1, bias=False),   # output 1 x 28 x 28
            nn.Tanh() # Normaliza a saída para [-1, 1]
        )

    def forward(self, z):
        return self.model(z)

#Observação: Adicionei bias=False nas camadas ConvTranspose2d do Gerador, uma prática comum em DCGANs quando se usa BatchNorm. A estrutura original do Gerador foi mantida.



```
⚙️ Etapa 6: Inicializar modelos, otimizadores e parâmetros
```



In [28]:
latent_dim = 100
# batch_size definido no DataLoader (64)
epochs = 100 # Pode precisar de ajuste; 100 é um bom começo
lr = 0.0002
beta1 = 0.5 # Beta1 recomendado para Adam em GANs
beta2 = 0.999

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

generator = Generator(latent_dim).to(device)
discriminator = Discriminator().to(device)

Usando dispositivo: cpu


In [29]:
# Inicialização de pesos (opcional, mas pode ajudar)
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

generator.apply(weights_init)
discriminator.apply(weights_init)

optimizer_G = optim.Adam(generator.parameters(), lr=lr, betas=(beta1, beta2))
optimizer_D = optim.Adam(discriminator.parameters(), lr=lr, betas=(beta1, beta2))
loss_fn = nn.BCELoss() # Binary Cross Entropy Loss

real_label = 1.
fake_label = 0.



```
🔁 Etapa 7: Treinar a GAN e Visualizar Resultados Parciais
```



In [ ]:
for epoch in range(epochs):
    for i, real_imgs in enumerate(dataloader):
        real_imgs = real_imgs.to(device)
        b_size = real_imgs.size(0)

        # --- Treinar o Discriminador ---
        optimizer_D.zero_grad()

        # Labels para imagens reais e falsas
        labels_real = torch.full((b_size,), real_label, dtype=torch.float, device=device)
        labels_fake = torch.full((b_size,), fake_label, dtype=torch.float, device=device)

        # Perda com imagens reais
        output_real = discriminator(real_imgs).view(-1)
        loss_real = loss_fn(output_real, labels_real)

        # Gerar imagens falsas
        noise = torch.randn(b_size, latent_dim, device=device)
        fake_imgs = generator(noise)

        # Perda com imagens falsas
        # Usar .detach() para não calcular gradientes para o gerador aqui
        output_fake = discriminator(fake_imgs.detach()).view(-1)
        loss_fake = loss_fn(output_fake, labels_fake)

        # Perda total do discriminador e otimização
        loss_D = loss_real + loss_fake
        loss_D.backward()
        optimizer_D.step()

        # --- Treinar o Gerador ---
        optimizer_G.zero_grad()

        # O gerador quer que o discriminador pense que as imagens falsas são reais
        labels_gen = torch.full((b_size,), real_label, dtype=torch.float, device=device)
        output_gen = discriminator(fake_imgs).view(-1) # Não usar .detach() aqui
        loss_G = loss_fn(output_gen, labels_gen)

        # Perda total do gerador e otimização
        loss_G.backward()
        optimizer_G.step()

    # Imprimir perdas ao final de cada época
    print(f"Epoch [{epoch+1}/{epochs}] | Loss_D: {loss_D.item():.4f} | Loss_G: {loss_G.item():.4f}")

    # Gerar e visualizar imagens (a cada 10 épocas)
    if (epoch + 1) % 10 == 0:
        with torch.no_grad(): # Não calcular gradientes durante a geração de amostras
            # Usar um tensor de ruído fixo para ver a evolução das mesmas amostras (opcional)
            # test_noise = getattr(generator, 'fixed_noise', torch.randn(16, latent_dim, device=device))
            # generator.fixed_noise = test_noise
            test_noise = torch.randn(16, latent_dim, device=device) # Ou gerar novo ruído sempre

            generated_imgs = generator(test_noise).cpu() # Mover para CPU para matplotlib

        # Ajustar visualização para o formato da imagem de saída do gerador (1, 28, 28)
        # As imagens estão em [-1, 1], reescale para [0, 1] para visualização se necessário ou imshow lida bem.
        # generated_imgs = (generated_imgs + 1) / 2 # Opcional: reescalar para [0,1]

        fig, axs = plt.subplots(4, 4, figsize=(4, 4))
        plt.suptitle(f"Época {epoch+1}", y=1.02) # Adicionar título com a época
        for k_ax, img_tensor in enumerate(generated_imgs):
            row, col = k_ax // 4, k_ax % 4
            ax = axs[row, col]
            ax.imshow(img_tensor.squeeze().numpy(), cmap='gray') # .squeeze() remove a dimensão do canal
            ax.axis('off')
        plt.tight_layout()
        plt.show()

Epoch [1/100] | Loss_D: 0.3602 | Loss_G: 1.9476
Epoch [2/100] | Loss_D: 0.4738 | Loss_G: 1.8353
Epoch [3/100] | Loss_D: 0.7217 | Loss_G: 1.5790
Epoch [4/100] | Loss_D: 0.8733 | Loss_G: 0.7849
Epoch [5/100] | Loss_D: 0.9333 | Loss_G: 1.1432
Epoch [6/100] | Loss_D: 1.1028 | Loss_G: 0.9215
Epoch [7/100] | Loss_D: 0.8960 | Loss_G: 1.3306




```
💾 Etapa 8: Salvar os modelos (após o treinamento)
```



In [ ]:
# Salvar os pesos dos modelos (state_dict)
torch.save(generator.state_dict(), "generator_digit4.pt")
torch.save(discriminator.state_dict(), "discriminator_digit4.pt")

print("Modelos salvos!")

# Para carregar os modelos posteriormente:
# generator = Generator(latent_dim)
# generator.load_state_dict(torch.load("generator_digit4.pt"))
# discriminator = Discriminator()
# discriminator.load_state_dict(torch.load("discriminator_digit4.pt"))
# generator.to(device)
# discriminator.to(device)
# generator.eval() # Para inferência
# discriminator.eval() # Para inferência